# External Validation: An Episode of Futurama

As an external validation exercise we will try to automatically detect gunshots in the futurama episode <a href=https://www.imdb.com/title/tt1630889/> "Law and Oracle".  <a/>

First we will load in the preprocessed data, split in ~5 second chunks.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Keys for classifier 

full_classify_keys = [
 'eq_0',
 'eq_10',
 'eq_20',
 'eq_30',
 'eq_40',
 'eq_60',
 'eq_80',
 'eq_120',
 'eq_160',
 'eq_230',
 'eq_300',
 'eq_450',
 'eq_600',
 'eq_900',
 'eq_1200',
 'eq_1800',
 'eq_2400',
 'eq_3700',
 'eq_5000',
 'eq_7500',
 'eq_10000',
 'eq_15000',
 'eq_20000',
 'crestfactor',
 #'salience', # We don't include this because it isn't in anything we can test on
 'harmonic_power',
 'percussive_power',
 'harmonic_hits',
 'percussive_hits',
 'power_ratio',
 'hits_ratio',
 'roll_32.spec_flatness.median',
 'roll_32.spec_flatness.iqr',
 'roll_32.spec_centroid.median',
 'roll_32.spec_bandwidth.median',
 'roll_32.spec_bandwidth.std',
 'roll_32.y_mw_zcr',
 'roll_64.spec_flatness.median',
 'roll_64.spec_flatness.iqr',
 'roll_64.spec_centroid.median',
 'roll_64.spec_bandwidth.median',
 'roll_64.spec_bandwidth.std',
 'roll_64.y_mw_zcr',
 'roll_128.spec_flatness.median',
 'roll_128.spec_flatness.iqr',
 'roll_128.spec_centroid.median',
 'roll_128.spec_bandwidth.median',
 'roll_128.spec_bandwidth.std',
 'roll_128.y_mw_zcr',
 'roll_64.logbands_mw_5.0',
 'roll_64.logbands_mw_5.1',
 'roll_64.logbands_mw_5.2',
 'roll_64.logbands_mw_5.3',
 'roll_64.logbands_mw_5.4',
 'roll_64.logbands_mw_10.0',
 'roll_64.logbands_mw_10.1',
 'roll_64.logbands_mw_10.2',
 'roll_64.logbands_mw_10.3',
 'roll_64.logbands_mw_10.4',
 'roll_64.logbands_mw_10.5',
 'roll_64.logbands_mw_10.6',
 'roll_64.logbands_mw_10.7',
 'roll_64.logbands_mw_10.8',
 'roll_64.logbands_mw_10.9',
 'roll_64.logbands_mw_20.0',
 'roll_64.logbands_mw_20.1',
 'roll_64.logbands_mw_20.2',
 'roll_64.logbands_mw_20.3',
 'roll_64.logbands_mw_20.4',
 'roll_64.logbands_mw_20.5',
 'roll_64.logbands_mw_20.6',
 'roll_64.logbands_mw_20.7',
 'roll_64.logbands_mw_20.8',
 'roll_64.logbands_mw_20.9',
 'roll_64.logbands_mw_20.10',
 'roll_64.logbands_mw_20.11',
 'roll_64.logbands_mw_20.12',
 'roll_64.logbands_mw_20.13',
 'roll_64.logbands_mw_20.14',
 'roll_64.logbands_mw_20.15',
 'roll_64.logbands_mw_20.16',
 'roll_64.logbands_mw_20.17',
 'roll_64.logbands_mw_20.18',
 'roll_64.logbands_mw_20.19']

In [3]:
# Only keys for the equalizer

eq_keys = [ 'eq_0',
 'eq_10',
 'eq_20',
 'eq_30',
 'eq_40',
 'eq_60',
 'eq_80',
 'eq_120',
 'eq_160',
 'eq_230',
 'eq_300',
 'eq_450',
 'eq_600',
 'eq_900',
 'eq_1200',
 'eq_1800',
 'eq_2400',
 'eq_3700',
 'eq_5000',
 'eq_7500',
 'eq_10000',
 'eq_15000',
 'eq_20000']

In [4]:
classify_keys = ['eq_0', 'eq_10', 'eq_20', 'eq_30', 'eq_40', 'eq_60', 'eq_80', 'eq_120', 'eq_160', 'eq_230', 'eq_300', 'eq_450', 'eq_600', 'eq_900', 'eq_1200', 'eq_1800', 'eq_2400', 'eq_3700', 'eq_5000', 'eq_7500', 'eq_10000', 'eq_15000', 'eq_20000', 'crestfactor', 'harmonic_power', 'percussive_power', 'harmonic_hits', 'percussive_hits', 'power_ratio', 'hits_ratio', 'roll_32.spec_flatness.median', 'roll_32.spec_flatness.iqr', 'roll_32.spec_centroid.median', 'roll_32.spec_bandwidth.median', 'roll_32.spec_bandwidth.std', 'roll_32.y_mw_zcr', 'roll_64.spec_flatness.median', 'roll_64.spec_flatness.iqr', 'roll_64.spec_centroid.median', 'roll_64.spec_bandwidth.median', 'roll_64.spec_bandwidth.std', 'roll_64.y_mw_zcr', 'roll_128.spec_flatness.median', 'roll_128.spec_flatness.iqr', 'roll_128.spec_centroid.median', 'roll_128.spec_bandwidth.median', 'roll_128.spec_bandwidth.std', 'roll_128.y_mw_zcr', 'roll_64.logbands_mw_5.1', 'roll_64.logbands_mw_5.2', 'roll_64.logbands_mw_5.3', 'roll_64.logbands_mw_10.1', 'roll_64.logbands_mw_10.2', 'roll_64.logbands_mw_10.3', 'roll_64.logbands_mw_10.4', 'roll_64.logbands_mw_10.5', 'roll_64.logbands_mw_10.6', 'roll_64.logbands_mw_10.7', 'roll_64.logbands_mw_10.8', 'roll_64.logbands_mw_20.0', 'roll_64.logbands_mw_20.2', 'roll_64.logbands_mw_20.3', 'roll_64.logbands_mw_20.4', 'roll_64.logbands_mw_20.5', 'roll_64.logbands_mw_20.6', 'roll_64.logbands_mw_20.7', 'roll_64.logbands_mw_20.8', 'roll_64.logbands_mw_20.9', 'roll_64.logbands_mw_20.10', 'roll_64.logbands_mw_20.11', 'roll_64.logbands_mw_20.12', 'roll_64.logbands_mw_20.13', 'roll_64.logbands_mw_20.14', 'roll_64.logbands_mw_20.15', 'roll_64.logbands_mw_20.16', 'roll_64.logbands_mw_20.17', 'roll_64.logbands_mw_20.18']

In [5]:
eq_df = pd.read_csv('large_data/ProcessedFuturamaValidationCombined.csv') # unsorted Equalizer and musical features with bad eq values
eq_correction = pd.read_csv('./large_data/futurama_eq_only.csv') # Just corrected eq values
eq_correction.fillna(0, inplace=True) # there's 1 NaN in there, because one of the vectors had 0 ultrasound elements. Replace by zero.
eq_df.sort_values(by="index", inplace=True, ignore_index=True) # sort the unsorted stuff based on the column labelled "index", replace in built index by sorted index

In [6]:
display(eq_df)

,index,eq_0,eq_10,eq_20,eq_30,eq_40,eq_60,eq_80,eq_120,eq_160,...,eq_5000,eq_7500,eq_10000,eq_15000,eq_20000,crestfactor,harmonic_power,percussive_power,harmonic_hits,percussive_hits
0,1,1.570806,2.268859,2.974112,4.025091,6.356958,7.247229,10.690657,13.122826,15.855344,...,11.958498,9.414774,6.484383,4.894141,4.440819,12.826445,4.715899e-07,0.000010,435.983945,13
1,2,10.211467,21.306899,19.720333,32.643308,107.079279,122.547807,132.923972,128.007142,117.933302,...,18.179263,14.953826,11.137923,8.402629,7.625240,5.730433,5.734467e-05,0.000032,173.944954,24
2,3,10.645605,20.278152,18.709405,31.525927,104.033239,117.089274,129.117981,124.778735,110.627750,...,16.092587,13.194110,9.920777,7.494981,6.801317,6.129410,9.824629e-05,0.000038,237.669789,20
3,4,8.689409,12.325558,12.257843,15.473838,31.126832,48.254391,58.571792,57.706232,57.310601,...,13.939281,11.397778,8.084987,6.138933,5.570131,9.856391,3.956560e-05,0.000036,138.429234,11
4,5,19.993627,18.715352,20.479738,21.420453,27.236543,34.125161,42.330845,44.963836,49.376026,...,16.344480,12.766403,8.825589,6.646938,6.030690,8.134315,9.298115e-05,0.000019,59.366972,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,256,6.511380,8.727036,11.813582,15.805200,72.941161,110.256949,125.107379,113.261637,97.888859,...,14.615230,12.308820,9.315662,7.061492,6.408703,7.007351,8.983630e-05,0.000016,210.802752,27
256,257,9.348906,19.944001,20.497543,42.978121,117.380167,130.733789,142.978155,134.900809,120.783368,...,16.067272,13.492240,10.376947,7.826219,7.100566,6.670063,5.429132e-05,0.000046,206.451991,23
257,258,11.353394,19.115732,18.947351,33.500692,114.085863,127.585604,139.364418,131.766075,115.424517,...,17.961738,14.935671,11.217921,8.457329,7.673414,6.649526,5.665979e-05,0.000038,278.233708,25
258,259,6.478106,10.290823,11.621443,24.280759,58.177907,59.662505,58.814831,53.372509,49.323425,...,8.856198,7.118954,4.970610,3.747288,3.400282,12.588661,1.077642e-05,0.000004,106.055690,14


In [7]:
eq_df['eq_0'] = eq_correction['eq_0'].replace(0,1e-16)
eq_df['eq_10'] = eq_correction['eq_10'].replace(0,1e-16)
eq_df['eq_20'] = eq_correction['eq_20'].replace(0,1e-16) # Set zeros to small number for regularization in log
eq_df['eq_30'] = eq_correction['eq_30'].replace(0,1e-16)
eq_df['eq_40'] = eq_correction['eq_40'].replace(0,1e-16)
eq_df['eq_60'] = eq_correction['eq_60'].replace(0,1e-16)
eq_df['eq_80'] = eq_correction['eq_80'].replace(0,1e-16)
eq_df['eq_120'] = eq_correction['eq_120'].replace(0,1e-16)
eq_df['eq_160'] = eq_correction['eq_160'].replace(0,1e-16)
eq_df['eq_230'] = eq_correction['eq_230'].replace(0,1e-16)
eq_df['eq_300'] = eq_correction['eq_300'].replace(0,1e-16)
eq_df['eq_450'] = eq_correction['eq_450'].replace(0,1e-16)
eq_df['eq_600'] = eq_correction['eq_600'].replace(0,1e-16)
eq_df['eq_900'] = eq_correction['eq_900'].replace(0,1e-16)
eq_df['eq_1200'] = eq_correction['eq_1200'].replace(0,1e-16)
eq_df['eq_1800'] = eq_correction['eq_1800'].replace(0,1e-16)
eq_df['eq_2400'] = eq_correction['eq_2400'].replace(0,1e-16)
eq_df['eq_3700'] = eq_correction['eq_3700'].replace(0,1e-16)
eq_df['eq_5000'] = eq_correction['eq_5000'].replace(0,1e-16)
eq_df['eq_7500'] = eq_correction['eq_7500'].replace(0,1e-16)
eq_df['eq_10000'] = eq_correction['eq_10000'].replace(0,1e-16)
eq_df['eq_15000'] = eq_correction['eq_15000'].replace(0,1e-16)
eq_df['eq_20000'] = eq_correction['eq_20000'].replace(0,1e-16)
eq_df.drop(columns='index',inplace=True)

In [8]:
features_equalizer = eq_df.copy()
features = pd.read_csv('large_data/futurama_engineered_features_full1.csv', index_col=0) # Additional features

In [9]:
features_df = pd.concat([features_equalizer, features], axis=1)

#features_df = features_frick
keys = list(features_df.keys())

In [10]:
# Scaling equalizer keys to bring them onto a more comparable scale

for eq in eq_keys:
    features_df[eq] = np.log10(features_df[eq])

In [11]:
# Define power and hits ratios between percussive and harmonic features

features_df['power_ratio'] = np.log10(features_df['percussive_power'].values / (features_df['harmonic_power'].values))
features_df['hits_ratio'] = np.log10(features_df['percussive_hits'].values / (features_df['harmonic_hits'].values + 1e-1) + 5e-4)

In [12]:
features_df.describe()

,eq_0,eq_10,eq_20,eq_30,eq_40,eq_60,eq_80,eq_120,eq_160,eq_230,...,roll_128.logbands_mw_20.12,roll_128.logbands_mw_20.13,roll_128.logbands_mw_20.14,roll_128.logbands_mw_20.15,roll_128.logbands_mw_20.16,roll_128.logbands_mw_20.17,roll_128.logbands_mw_20.18,roll_128.logbands_mw_20.19,power_ratio,hits_ratio
count,260.000000,260.000000,260.000000,260.000000,260.000000,260.000000,260.000000,260.000000,260.000000,260.000000,...,260.000000,260.000000,260.000000,260.000000,260.000000,260.000000,260.000000,260.0,260.000000,260.000000
mean,0.314869,0.303962,0.411564,0.678951,0.968327,1.086938,1.193811,1.391447,1.550832,1.630000,...,0.908751,0.873822,0.826302,0.767427,0.663718,0.514354,0.339957,0.0,0.328113,-1.183607
std,0.626096,0.740232,0.770764,0.810627,0.637689,0.636969,0.577675,0.477285,0.305965,0.279498,...,0.019232,0.021837,0.024566,0.030679,0.037776,0.048535,0.037067,0.0,1.177486,0.394230
min,-0.567769,-0.960618,-0.818749,-0.780842,-0.439494,-0.287441,-0.263382,0.158898,0.521616,0.603225,...,0.857288,0.813768,0.753002,0.675868,0.577964,0.393316,0.209180,0.0,-2.802040,-3.301030
25%,-0.212212,-0.322561,-0.229626,0.043576,0.553384,0.565708,0.685231,1.054539,1.372989,1.470992,...,0.893899,0.858785,0.808896,0.746867,0.638408,0.484640,0.319099,0.0,-0.389130,-1.437599
50%,0.093561,0.143516,0.210541,0.373672,0.705489,0.898326,1.183641,1.406870,1.552626,1.642765,...,0.908488,0.874603,0.829323,0.767245,0.663776,0.523363,0.342395,0.0,0.387965,-1.177538
75%,0.823149,0.982701,1.144887,1.355772,1.486209,1.585776,1.632487,1.740037,1.742769,1.796408,...,0.923653,0.890839,0.844596,0.787572,0.688080,0.550046,0.362634,0.0,1.041902,-0.921934
max,1.951010,2.095531,2.216003,2.516181,2.611051,2.473794,2.483778,2.355153,2.397059,2.599803,...,0.960451,0.922214,0.883592,0.867104,0.782336,0.629004,0.428980,0.0,4.909115,-0.145119


In [13]:
# Encode classes into numerical values
features_df.replace({'air_conditioner':0, 'car_horn':1, 'children_playing':2, 'dog_bark':3, 'drilling':4,
                'engine_idling':5, 'gun_shot':6, 'jackhammer':7, 'siren':8, 'street_music':9},inplace=True)

In [14]:
classlabels = {'air_conditioner':0, 'car_horn':1, 'children_playing':2, 
               'dog_bark':3, 'drilling':4, 'engine_idling':5, 
               'gun_shot':6, 'jackhammer':7, 'siren':8, 'street_music':9}.keys()

In [15]:
save = False
if save:
    features_df.to_csv('futurama_features_df.csv')

In [16]:
np.where(np.isnan(features_df[full_classify_keys].values) == 1)

(array([], dtype=int64), array([], dtype=int64))

We then import the pickled (i.e. saved) models for the logistic regression and random forest. From here we will pass the data extracted from the futurama episode through the prediction function and analyze which clips, by index, are identified as gunshots.

In [17]:
import pickle

In [18]:
forest = pickle.load(open('fullfeaturesRobustScaledRandomForest.pkl','rb'))
logistic = pickle.load(open('fullfeaturesRobustScaledlogistic.pkl','rb'))

/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RobustScaler from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.0.2. This m

In [19]:
X_test = features_df[classify_keys].values

In [23]:
forest_pred = forest.predict(X_test)
logistic_pred = logistic.predict(X_test)

In [24]:
forest_pred

array([2, 7, 7, 7, 9, 9, 3, 2, 9, 9, 2, 2, 2, 3, 3, 2, 2, 9, 2, 6, 6, 2,
       9, 2, 2, 2, 6, 2, 9, 9, 9, 9, 9, 9, 3, 2, 2, 9, 9, 9, 9, 9, 9, 9,
       9, 2, 3, 9, 9, 9, 9, 9, 9, 9, 9, 9, 2, 9, 2, 9, 9, 3, 2, 3, 2, 9,
       9, 2, 9, 9, 2, 2, 3, 2, 9, 9, 9, 9, 9, 9, 6, 9, 9, 6, 9, 9, 9, 9,
       9, 9, 7, 9, 3, 9, 9, 2, 2, 3, 9, 9, 9, 3, 9, 3, 9, 9, 7, 7, 9, 2,
       2, 2, 6, 9, 9, 2, 2, 9, 2, 9, 9, 9, 2, 2, 2, 2, 2, 2, 2, 9, 2, 2,
       2, 9, 7, 2, 9, 3, 2, 2, 2, 9, 2, 9, 9, 9, 2, 2, 2, 9, 6, 9, 9, 9,
       9, 2, 9, 9, 9, 2, 9, 9, 2, 9, 9, 9, 2, 9, 9, 2, 9, 9, 3, 2, 2, 9,
       9, 9, 2, 2, 2, 2, 9, 9, 9, 9, 2, 3, 2, 9, 2, 2, 2, 2, 2, 9, 2, 2,
       9, 9, 3, 2, 9, 2, 9, 9, 9, 6, 2, 7, 4, 7, 3, 9, 2, 2, 2, 3, 9, 2,
       9, 2, 9, 9, 2, 9, 3, 2, 9, 2, 9, 6, 9, 9, 2, 9, 2, 2, 9, 2, 2, 9,
       2, 9, 2, 9, 3, 9, 9, 2, 2, 9, 2, 7, 7, 7, 7, 7, 9, 9])

In [25]:
logistic_pred

array([4, 9, 5, 4, 6, 0, 2, 2, 9, 4, 2, 9, 2, 3, 2, 3, 8, 8, 2, 3, 2, 2,
       9, 2, 2, 8, 3, 3, 2, 6, 9, 0, 2, 9, 9, 2, 2, 9, 9, 9, 9, 2, 9, 2,
       9, 9, 8, 9, 9, 4, 9, 9, 9, 6, 9, 9, 9, 8, 2, 3, 9, 8, 2, 3, 2, 2,
       9, 8, 9, 9, 2, 9, 3, 2, 9, 9, 9, 9, 9, 9, 9, 5, 9, 8, 9, 9, 9, 9,
       9, 9, 1, 9, 2, 8, 9, 9, 6, 8, 8, 8, 9, 3, 9, 3, 9, 9, 7, 4, 3, 3,
       2, 2, 2, 9, 9, 2, 2, 9, 2, 9, 3, 9, 2, 8, 8, 8, 2, 9, 2, 9, 2, 2,
       3, 6, 2, 6, 9, 2, 2, 2, 2, 9, 2, 3, 2, 9, 6, 2, 2, 9, 6, 9, 9, 9,
       9, 8, 9, 9, 2, 2, 9, 9, 9, 3, 9, 9, 3, 9, 9, 8, 9, 9, 2, 2, 2, 2,
       2, 6, 2, 3, 3, 8, 9, 9, 9, 2, 2, 8, 2, 3, 2, 2, 2, 2, 2, 9, 2, 2,
       2, 2, 2, 3, 9, 9, 9, 2, 8, 2, 2, 3, 9, 0, 3, 6, 9, 2, 2, 2, 9, 8,
       9, 2, 3, 8, 8, 8, 2, 2, 9, 8, 2, 3, 9, 6, 8, 2, 2, 2, 8, 2, 2, 9,
       2, 3, 8, 6, 9, 9, 9, 2, 8, 9, 2, 4, 9, 9, 5, 5, 9, 3])

In [32]:
np.where(forest_pred == 6)

(array([ 19,  20,  26,  80,  83, 112, 150, 207, 231]),)

In [27]:
np.where(logistic_pred == 6)

(array([  4,  29,  53,  96, 133, 135, 146, 150, 177, 213, 233, 245]),)

In [36]:
forest_pred[45:55]

array([2, 3, 9, 9, 9, 9, 9, 9, 9, 9])

In [35]:
logistic_pred[45:55]

array([9, 8, 9, 9, 4, 9, 9, 9, 6, 9])